In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, precision_score, confusion_matrix, f1_score, ConfusionMatrixDisplay
from sklearn.linear_model import LinearRegression 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import joblib

In [62]:
df = pd.read_csv("consumo_casa.csv")


df['Fecha'] = pd.to_datetime(df['Fecha'])
df['Dia'] = df['Fecha'].dt.day
df['Hora'] = df['Fecha'].dt.hour
df['Minuto'] = df['Fecha'].dt.minute

del df['Unnamed: 0']
#del df['Fecha']

columnas = df.columns.tolist()
ultimas_tres_columnas = columnas[-3:]
print(columnas[0])
nuevas_columnas = [columnas[0]] + ultimas_tres_columnas + columnas[1:-3]
df = df[nuevas_columnas]

df.columns = df.columns.str.replace(' ', '')
df = df.rename(columns={'Medidor[W]': 'Medidor'})
df1 = df.iloc[:, :5] # Selecciona las dos primeras columnas
df2 = df.iloc[:, 5:] # Selecciona las columnas desde la tercera en adelante
df1

Fecha


,Fecha,Dia,Hora,Minuto,Medidor
0,2017-06-07 00:00:00,7,0,0,228.887
1,2017-06-07 00:01:00,7,0,1,231.814
2,2017-06-07 00:02:00,7,0,2,230.971
3,2017-06-07 00:03:00,7,0,3,231.878
4,2017-06-07 00:04:00,7,0,4,189.225
...,...,...,...,...,...
194118,2017-10-22 23:55:00,22,23,55,88.272
194119,2017-10-22 23:56:00,22,23,56,86.386
194120,2017-10-22 23:57:00,22,23,57,85.766
194121,2017-10-22 23:58:00,22,23,58,87.665


### Clasificacion

In [63]:
df2c = df2.applymap(lambda x: 1 if x > 0 else x)
dfResultC = df1.copy()

C:\Users\Asus\AppData\Local\Temp\ipykernel_77148\2056342280.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2c = df2.applymap(lambda x: 1 if x > 0 else x)


In [57]:
def modelosPorDispositivo(index):
    X = df1.iloc[:,1:]
    y = df2c.iloc[:, index]
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)
    #X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.25,random_state=0) # 0.25 * 0.8 = 0.2
    Results = df1.copy()

    model_preds = []
    def fit_model(model, model_name):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matriz = confusion_matrix(y_test, y_pred)
        exactitud = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        disp = ConfusionMatrixDisplay(matriz)
        model_preds.append([model_name, exactitud, precision, f1])
        #disp.plot()
        #print ("Confusion Matrix: ", matriz)
        #print ("Accuracy Score: ", exactitud)
        #print("Precision Score: ", precision)
        #print("F1 Score: ", f1)

    def model_eval():
        preds = pd.DataFrame(model_preds)
        preds.columns = ["Mod_Name", "Accuracy", "Precision", "F1 Score"]
        return preds.sort_values(by="Precision", ascending=False)
    
    

    DTree_c = DecisionTreeClassifier(random_state=0)
    fit_model(DTree_c, "Dtree")
    
    joblib.dump(DTree_c, f'modelo_clasificacion_d{index}.pkl') # Guardo el modelo.

In [58]:
for i in range(8):
    modelosPorDispositivo(i)

In [59]:
modc0 = joblib.load('modelo_clasificacion_d0.pkl')
modc1 = joblib.load('modelo_clasificacion_d1.pkl')
modc2 = joblib.load('modelo_clasificacion_d2.pkl')
modc3 = joblib.load('modelo_clasificacion_d3.pkl')
modc4 = joblib.load('modelo_clasificacion_d4.pkl')
modc5 = joblib.load('modelo_clasificacion_d5.pkl')
modc6 = joblib.load('modelo_clasificacion_d6.pkl')
modc7 = joblib.load('modelo_clasificacion_d7.pkl')

In [60]:
modelos = {
    "modc0" : modc0,
    "modc1" : modc1,
    "modc2" : modc2,
    "modc3" : modc3,
    "modc4" : modc4,
    "modc5" : modc5,
    "modc6" : modc6,
    "modc7" : modc7,
}

In [72]:
X = df1.iloc[:,1:]
def predicciones(X):
    for i in range(8):
        y_pred = modelos[f"modc{i}"].predict(X)
        dfResultC[f"Dispositivo{i}"] = y_pred

In [73]:
predicciones(X)
dfResultC

,Fecha,Dia,Hora,Minuto,Medidor,Dispositivo0,Dispositivo1,Dispositivo2,Dispositivo3,Dispositivo4,Dispositivo5,Dispositivo6,Dispositivo7
0,2017-06-07 00:00:00,7,0,0,228.887,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
1,2017-06-07 00:01:00,7,0,1,231.814,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
2,2017-06-07 00:02:00,7,0,2,230.971,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,2017-06-07 00:03:00,7,0,3,231.878,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
4,2017-06-07 00:04:00,7,0,4,189.225,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
194118,2017-10-22 23:55:00,22,23,55,88.272,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
194119,2017-10-22 23:56:00,22,23,56,86.386,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
194120,2017-10-22 23:57:00,22,23,57,85.766,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
194121,2017-10-22 23:58:00,22,23,58,87.665,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
